In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Preprocessing**

**feature**

In [ ]:
df_feature = pd.read_csv('../input/credit-card-approval-prediction/application_record.csv',header=0)
df_feature

In [ ]:
df_feature.describe()

In [ ]:
df_feature.isnull().any()

In [ ]:
df_feature.groupby('OCCUPATION_TYPE')['OCCUPATION_TYPE'].count()

In [ ]:
df_feature_null = df_feature.dropna(axis=0)
df_feature_null.isnull().any()

In [ ]:
df_feature_drop = df_feature_null.drop(['CODE_GENDER','DAYS_BIRTH','DAYS_EMPLOYED','FLAG_MOBIL','FLAG_WORK_PHONE','FLAG_EMAIL'],axis=1)
df_feature_drop.isnull().any()

In [ ]:
df_feature_clean = df_feature_drop
df_feature_clean.dtypes

In [ ]:
df_feature_clean['FLAG_OWN_CAR'] = pd.factorize(df_feature_clean['FLAG_OWN_CAR'])[0]
df_feature_clean['FLAG_OWN_REALTY'] = pd.factorize(df_feature_clean['FLAG_OWN_REALTY'])[0]
df_feature_clean['NAME_INCOME_TYPE'] = pd.factorize(df_feature_clean['NAME_INCOME_TYPE'])[0]
df_feature_clean['NAME_EDUCATION_TYPE'] = pd.factorize(df_feature_clean['NAME_EDUCATION_TYPE'])[0]
df_feature_clean['NAME_FAMILY_STATUS'] = pd.factorize(df_feature_clean['NAME_FAMILY_STATUS'])[0]
df_feature_clean['NAME_HOUSING_TYPE'] = pd.factorize(df_feature_clean['NAME_HOUSING_TYPE'])[0]
df_feature_clean['OCCUPATION_TYPE'] = pd.factorize(df_feature_clean['OCCUPATION_TYPE'])[0]
df_feature_clean.dtypes

In [ ]:
#ดึง ID ออกมาพื่อเอาไปเชื่อมกับของ label
feature_id = df_feature_clean['ID'].values
df_feature_clean = df_feature_clean.drop(['ID'],axis=1)

In [ ]:
feature_id

**Label**

In [ ]:
df_label = pd.read_csv('../input/credit-card-approval-prediction/credit_record.csv',header=0)
df_label

In [ ]:
df_label.groupby('ID')['STATUS'].count() #แต่ละคนมีกี่ record

In [ ]:
df_label['STATUS'], labels = pd.factorize(df_label['STATUS']) #เปลี่ยน status ให้เป็นเลข แล้วเก็บค่าเดิมไว้ในตัวแปรชื่อ labels
df_label

In [ ]:
labels #เรียงตาม index เป็นค่านี้ใน column ของ STATUS

<p> index 0 = X: No loan for the month
<p> index 1 = 0: 1-29 days past due 
<p> index 2 = C: paid off that month 
<p> index 3 = 1: 30-59 days past due 
<p> index 4 = 2: 60-89 days overdue 
<p> index 5 = 3: 90-119 days overdue  
<p> index 6 = 4: 120-149 days overdue 
<p> index7 = 5: Overdue or bad debts, write-offs for more than 150 days 

In [ ]:
df_label_clean_records = df_label[df_label['STATUS'] != 0] #ตัดข้อมูล X คือ เดือนนั้นไม่ได้กู้ออก (index0) เพราะไม่มีผลในการวิเคราะห์
df_label_clean_records

In [ ]:
df_label_clean_records['STATUS'][df_label_clean_records['STATUS']>5] = 10 #ผู้ที่จ่ายเงินเกินกำหนดในแต่ละ record = 10
df_label_clean_records['STATUS'][df_label_clean_records['STATUS']<=5] = 0 #ผู้ที่จ่ายเงินตรงและเกินเพียง 1 เดือนในแต่ละ record = 0     
df_label_clean_records

In [ ]:
df_label_clean_records.groupby('STATUS')['STATUS'].count()

In [ ]:
df_label_clean = df_label_clean_records.groupby('ID').mean() #หาค่า avg status ของแต่ละ ID
df_label_clean

เป็น 0 แปลว่าจ่ายตรงทุกครั้ง เพราะ เฉลี่ย = 0

In [ ]:
df_label_clean['STATUS'][df_label_clean['STATUS'] > 0 ] = 1 #ID ที่จ่ายไม่ตรง bad debt = 1
df_label_clean['STATUS'][df_label_clean['STATUS'] <= 0 ] = 0 #ID ที่จ่ายตรงตลอด good debt = 0
df_label_clean

In [ ]:
df_label_clean.groupby('ID')['STATUS'].count() #Length: 41449

In [ ]:
df_label_clean.groupby('STATUS')['STATUS'].count() #41118+331 = 41449

In [ ]:
#ดึงข้อมูลที่ต้องใช้ใน label ออกมา
label_id = df_label_clean.index.values #เอา ID (ที่ถูก set ให้เป็น index) ใน label ออกมา เพื่อ ไปเชื่อมกับ feature
label_status = df_label_clean['STATUS'].values #เอา status ใน label ออกมา เพื่อใช้เป็น label ของ model

In [ ]:
label_id

In [ ]:
label_status

**check feature & label**

In [ ]:
ID = [ item for item in feature_id if item in label_id] #เช็ค id จาก feature และ labelที่เหมือนกัน
len(ID)

In [ ]:
#feature ของ model
features = df_feature_clean.values
X_list = [ features[np.where(feature_id == item), :][0] for item in ID ] #เอา feature จริงๆ ตรงกับ ID ที่เชื่อมกันออกมา
X = np.vstack(X_list) #เอา list มาต่อกันให้เป็น np.array
X.shape

In [ ]:
#label ของ model
y = np.array([ label_status[np.where(label_id == item)][0] for item in ID ]) #เอา Label จริงๆ ตรงกับ ID ที่เชื่อมกันออกมา
y.shape

In [ ]:
sum(y==0)

In [ ]:
sum(y==1)

# **Model**

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)

In [ ]:
#แก้ imblance โดยการใช้ undersample มาเท่ากับ 3089(y=1), ใช้ undersampling เพราะข้อมูลเยอะมาก
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority', random_state=1) #resample of imbalanced label
X_res, y_res = undersample.fit_resample(X_train, y_train)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_res, y_res,test_size=0.2,random_state=0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm =  scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 2, 4, 8, 16, 32, 64]}
clf = GridSearchCV(LogisticRegression(random_state=0, solver='liblinear'),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

In [ ]:
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
target_names = ['Good debt', 'Bad debt']
C = confusion_matrix(y_test,y_predict) 
C = C / C.astype(np.float).sum(axis=1)*100
sns.heatmap(C,annot=True, fmt=".2f",cmap="GnBu",xticklabels=target_names, yticklabels=target_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict,target_names=target_names))

**Deep learning**

In [ ]:
import tensorflow as tf
def create_model():
    tf.random.set_seed(0)
    tf.compat.v1.reset_default_graph() # Clear Model
    model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(12, activation='relu', input_shape=(11,)),
      tf.keras.layers.Dense(6, activation='relu'),
      tf.keras.layers.Dense(4, activation='relu'),
      tf.keras.layers.Dense(2, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')  
    ])
    return model

In [ ]:
model = create_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

#ใช้ algorithm ชื่อ adam
#ใช้ binary_crossentropy เพราะคำตอบของ label มีแค่ 2 ค่า

Training

In [ ]:
train_acc = list()
val_acc = list()
for i in range(0,140):
  history = model.fit(X_train_norm, y_train, epochs= 1, batch_size = 300, validation_data= (X_val_norm, y_val))
  tmp_avg = np.mean(history.history['accuracy'])
  tmp_avg_val = np.mean(history.history['val_accuracy'])
  train_acc.append(tmp_avg)
  val_acc.append(tmp_avg_val)

#batch_size คือ เอาข้อมูลเข้าไป train ทีละเท่าไหร่

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.figure(num=None, figsize=(16, 8), dpi=90, facecolor='w', edgecolor='k')
plt.plot()
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Testing

In [ ]:
y_predict = np.round(model.predict(X_test_norm))
y_predict = [i[0] for i in y_predict.tolist()]
sum(y_predict == y_test)/len(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
target_names = ['Good debt', 'Bad debt']
C = confusion_matrix(y_test,y_predict) 
C = C / C.astype(np.float).sum(axis=1)*100
sns.heatmap(C, annot=True, fmt=".2f",cmap="GnBu",xticklabels=target_names, yticklabels=target_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict,target_names=target_names))